In [1]:
import os

from dotenv import load_dotenv

if os.getcwd().endswith("notebooks"):
    os.chdir("./..")
print("Current Directory:", os.getcwd())
load_dotenv()

Current Directory: /Users/rafaelmadrigal/Documents/Code-Work/pluto/project-pluto-2/project-pluto


True

In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("./data/transactions.db")
display(pd.read_sql("""SELECT * FROM transactions""", conn))


,user_id,transaction_month,transaction_year,transaction_frequency,transaction_total_amount,product_code,event_code,transaction_type
0,217020000647673395295,01,2019,212,4194.21,51051000101200000031,12128004,DEBIT
1,217010000649368875922,01,2019,7,1733.88,51051000101000600001,12128026,CREDIT
2,217010000649065802481,01,2019,35,3335.07,51052001000000000132,12149303,CREDIT
3,217020000648021944677,01,2019,3,5600.01,51051000101000200040,12128004,DEBIT
4,217020000647463627919,01,2019,3,2285.12,51090000001432700001,12128004,DEBIT
...,...,...,...,...,...,...,...,...
17995,217010000648943289772,06,2024,16,1957.85,51051000101121000062,12128004,DEBIT
17996,217010000648085627045,06,2024,5,1863.60,51052001000000000015,12149302,DEBIT
17997,217010000649490940545,06,2024,1,2117.92,51051000100000000088,12149302,CREDIT
17998,217010000649335145920,06,2024,1,2373.28,51052001000000000022,31020303,DEBIT


In [3]:
from bot.pluto import build_pluto_chain
from bot.sqldb import get_db
from bot.utils import get_llm_chat_model
from bot.prompts import (
    DESCRIPTION__SPEND_TRACKER_AGGREGATIONS,
    DESCRIPTION__SPEND_TRACKER_CATEGORIES,
)

llm = get_llm_chat_model(endpoint="openai", model_name="gpt-3.5-turbo", temperature=0)

sql_db = get_db(
    "transactions.db",
    "./data",
    additional_table_info=dict(
        spend_tracker_aggregations=DESCRIPTION__SPEND_TRACKER_AGGREGATIONS,
        spend_tracker_categories=DESCRIPTION__SPEND_TRACKER_CATEGORIES,
    ),
)

DB Dialect: sqlite
DB Usable Table Names: ['spend_tracker_aggregations', 'spend_tracker_categories']


In [43]:
q1 = dict(question="What are my Request Money Payment transactions in the 1st quarter?")
result = pluto_chain.invoke(q1)
print("-" * 100)
print("QUESTION:")
print(q1)
print("ANSWER:")
print(result["answer"])
print("QUERY:")
print(result["sql_query"])

----------------------------------------------------------------------------------------------------
QUESTION:
{'question': 'What are my Request Money Payment transactions in the 1st quarter?'}
ANSWER:
Hey there! Your Request Money Payment transactions in the 1st quarter amounted to a total of PHP 2,000,000.13. It looks like you had quite a number of transactions during that period. If you need more details or have any other questions, feel free to ask!
QUERY:
SELECT "transaction_total_amount", "transaction_type", "usecase", "category"
FROM "spend_tracker_aggregations" AS sta
JOIN "spend_tracker_categories" AS stc ON sta."product_code" = stc."product_code" AND sta."event_code" = stc."event_code"
WHERE "transaction_type" = 'DEBIT' 
AND "category" = 'Request Money Payment' 
AND "transaction_month" IN ('01', '02', '03')
AND "transaction_year" = (CASE 
                            WHEN CAST(strftime('%m', date('now')) AS INTEGER) >= 4 THEN strftime('%Y', date('now'))
                       